In [1]:
#export
from pytex.Variable import Var, makeVar

class Matrix(Var):
    def __init__(self, matrix=[[]], subscript=None, power=None, surround='[]'):
        super().__init__('matrix')
        self.matrix, self.surround = matrix, surround
        self.power = makeVar(power) if power is not None else power
        self.subscript = makeVar(subscript) if subscript is not None else subscript
        
    def __len__(self):
        return self.rows
    
    def __getitem__(self, ind):
        if ind  >= len(self.matrix): raise Exception('index out of bounds')
        if len(self.matrix) == 0: raise Exception('empty matrix') 
        if isinstance(self.matrix[0], list): return Matrix(self.matrix[ind])
        else: return Var(self.matrix[ind])
    
    @staticmethod
    def builder():
        return MatrixBuilder()
        
    @property
    def T(self):
        if len(self.matrix) == 0: return self
        elif(not isinstance(self.matrix[0], list)):
            rows, cols = len(self.matrix), 1
            matrix = [[ele] for ele in self.matrix]
        else:
            rows, cols = len(self.matrix), len(self.matrix[0])
            matrix = [[self.matrix[j][i] for j in range(rows)] for i in range(cols)]
        return Matrix(matrix, self.subscript, self.power, self.surround)
        
    def build(self): 
        env = (r'{bmatrix}' if self.surround == '[]' 
               else r'{pmatrix}' if self.surround == '()'
               else r'{vmatrix}' if self.surround == '||'
               else r'{Vmatrix}' if (self.surround == '|| ||' or self.surround == '||||')
               else r'{matrix}')
        begin = r'\begin' + env
        end = r'\end' + env
        if len(self.matrix) == 0: return begin + end
        
        s = ''
        if not isinstance(self.matrix[0], list):
            # it is a vector
            for ele in self.matrix:
                ele = makeVar(ele)
                s += ' ' + ele.build() + ' & '
            s = s[:-2]
        else:
            for row in self.matrix:
                for ele in row:
                    ele = makeVar(ele)
                    s += ' ' + ele.build() + ' & '
                s = s[:-2] + r"\\"
            s = s[:-2]
        p = '^{' + self.power.build() + '}' if self.power is not None else ''
        subs = '_{' + self.subscript.build() + '}' if self.subscript is not None else ''
        return (r"" + begin + s + end  + p + subs + r"")


In [2]:
#export
class MatrixBuilder():
    def __init__(self):
        self.matrix, self.cols = [], None
        
    def add(self, *args):
        if self.cols is None:
            self.cols = len(args)
        elif not self.cols == len(args):
            raise Exception(f'Different dimensions for matrix builder. dim1:{self.cols} dim2:{len(args)}')
        self.matrix.append(list(args))
        return self
        
    def create(self, subscript=None, power=None, surround='[]'):
        return Matrix(self.matrix, subscript, power, surround)

In [3]:
#export
from pytex.Dots import Dots
def hasHoriDots(elements, m):
    elerowlen = max([len(ele) for ele in elements]) 
    return not elerowlen == len(m[0])
def hasVertDots(elements, m):
    return not len(elements) == len(m)

class MatrixWithDots(Matrix):
    def __init__(self, elements, shape=None, subscript=None, power=None, surround='[]'):
        # resolve elements
        self.elements, self.shape = elements,shape
        if(not isinstance(elements, list) or len(elements) == 0
          or not isinstance(elements[0], list)): 
            raise Exception('need to pass list of list')
        # resolve shape
        minrows, mincols = len(elements), max([len(row) for row in elements])
        if(shape is None):
            self.rows = minrows + 1 if minrows != 1 else minrows
            self.cols = mincols + 1 if mincols != 1 else mincols
        else:
            try:
                self.rows, self.cols = shape
                self.rows, self.cols = int(self.rows), int(self.cols)
            except:
                raise Exception('shape should be of type tuple(int,int)')
            if self.rows < minrows or self.cols < mincols:
                raise Exception('shape >= (row, cols) of elements')
        m = [[0]*self.cols for i in range(self.rows)]
        i=0
        while i < len(elements)-1:
            row = elements[i]
            j=0
            while(j < len(row)-1):
                m[i][j] = row[j]
                j += 1
            while(j < len(m[i])-1):
                m[i][j] = Dots('h')
                j += 1
            m[i][j] = row[len(row)-1]
            i += 1

        while i < len(m)-1:
            m[i] = [Dots('v')]*len(m[i])
            i += 1

        row = elements[len(elements)-1]
        j=0
        while(j < len(row)-1):
            m[i][j] = row[j]
            j += 1
        while(j < len(m[i])-1):
            m[i][j] = Dots('h')
            j += 1
        m[i][j] = row[len(row)-1]

        dli, dlj = len(m)-2, len(m[0])-2

        if isinstance(m[dli][dlj], Dots) and hasHoriDots(elements, m) and hasVertDots(elements, m):
            m[dli][dlj] = Dots('d')
        super().__init__(m, subscript, power, surround)
        
    @property
    def T(self):
        raise Exception('transpose of dotted matrix not allowed')

In [4]:
from pytex.platforms.jupyter import latex

latex(MatrixWithDots([[1,2,3,4],[2,3,4,5]]))

<IPython.core.display.Latex object>

In [5]:
latex(Matrix([[1,2,3,4],[2,3,4,5]]))

<IPython.core.display.Latex object>

In [6]:
latex(Matrix.builder().add(1,2,3,4).add(2,3,4,5).create())

<IPython.core.display.Latex object>

In [8]:
! python3 ./notebook2script.py Matrix.ipynb

Converted /home/default/Documents/py-latex/pytex/notebooks/Matrix.ipynb to /home/default/Documents/py-latex/pytex/pytex/Matrix.py
